In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# Load in data
projectfolder = '/User/Symptoms_Documentation/'

df_match = pd.read_csv(projectfolder + 'data/reviewer_match.csv')
df_notes = pd.read_excel(projectfolder + 'data/second_reviewer.xlsx')
df_notes = df_notes.rename(columns={'Study ID ': 'Study ID', 'Unnamed: 1': 'Note ID'})

display(df_match)
display(df_notes)

In [ ]:
# total number of instances that both raters said were correct
A = np.sum(df_match['Rater1'] & df_match['Rater2'])
# total number of instances rater 2 said was incorrect but rater 1 said was correct
B = np.sum(df_match['Rater1'] & ~df_match['Rater2'])
# total number of instances rater 1 said was incorrect but rater 2 said was correct
C = np.sum(~df_match['Rater1'] & df_match['Rater2'])
# total number of instances both raters said were incorrect
D = np.sum(~df_match['Rater1'] & ~df_match['Rater2'])

# probability of agreement
p_a = (A+D) / (A+B+C+D)
p_correct = ((A+B)/(A+B+C+D)) * ((A+C)/(A+B+C+D))
p_incorrect = ((C+D)/(A+B+C+D)) * ((B+D)/(A+B+C+D))
p_e = p_correct + p_incorrect

print(A, B, C, D, p_a, p_correct, p_incorrect, p_e)

kappa = (p_a - p_e) / (1-p_e)
print('kappa:', kappa)

In [ ]:
# Load data
df_match_v2 = pd.read_csv(projectfolder + 'data/reviewer_match_corrected.csv')
display(df_match_v2)

In [ ]:
# total number of instances that both raters said were correct
A = np.sum(df_match_v2['Rater1 corrected'] & df_match_v2['Rater2 corrected'])
# total number of instances rater 2 said was incorrect but rater 1 said was correct
B = np.sum(df_match_v2['Rater1 corrected'] & ~df_match_v2['Rater2 corrected'])
# total number of instances rater 1 said was incorrect but rater 2 said was correct
C = np.sum(~df_match_v2['Rater1 corrected'] & df_match_v2['Rater2 corrected'])
# total number of instances both raters said were incorrect
D = np.sum(~df_match_v2['Rater1 corrected'] & ~df_match_v2['Rater2 corrected'])

# probability of agreement
p_a = (A+D) / (A+B+C+D)
p_correct = ((A+B)/(A+B+C+D)) * ((A+C)/(A+B+C+D))
p_incorrect = ((C+D)/(A+B+C+D)) * ((B+D)/(A+B+C+D))
p_e = p_correct + p_incorrect

print(A, B, C, D, p_a, p_correct, p_incorrect, p_e)

kappa = (p_a - p_e) / (1 - p_e)
print('kappa:', kappa)

In [ ]:
# Extract Study IDs fromt he dataframe
sample_ids = df_match_v2['Study ID']

# List of Study IDs where symptoms were detected using NLP
cramps_nlp_all = [1, 3, 9, 11, 16, 18, 19, 20, 34, 36, 38, 50, 51, 54, 56, 57, 60, 65, 69, 70, 73, 76, 81, 82, 84, 100]
dryskin_nlp_all = []
fatigue_nlp_all = [8, 19, 33, 39, 42, 49, 61, 62, 75, 78, 79, 81, 89, 93]
# itching_nlp_all = [2, 3, 6, 9, 10, 11, 16, 31, 37, 38, 47, 54, 60, 62, 66, 75, 77, 83, 97]
itching_nlp_all = [2, 3, 6, 9, 10, 11, 16, 31, 37, 38, 47, 54, 60, 62, 66, 75, 77, 83, 97, 100]
musclesore_nlp_all = [1, 42, 66, 73, 76, 86, 88, 94, 95]

# Filter actual data for each symptom (confirmed by Rater2) and get Study IDs
true_filt = df_match_v2['Rater2 corrected']
cramps_actual = df_match_v2.loc[true_filt & (df_match_v2['Symptom']=='Cramps'), 'Study ID'].to_list()
dryskin_actual = df_match_v2.loc[true_filt & (df_match_v2['Symptom']=='Dry skin'), 'Study ID'].to_list()
fatigue_actual = df_match_v2.loc[true_filt & (df_match_v2['Symptom']=='Fatigue'), 'Study ID'].to_list()
itching_actual = df_match_v2.loc[true_filt & (df_match_v2['Symptom']=='itching'), 'Study ID'].to_list()
musclesore_actual = df_match_v2.loc[true_filt & (df_match_v2['Symptom']=='Muscle Soreness'), 'Study ID'].to_list()

# Filter NLP-Detected IDs to include only those in the sample
cramps_nlp = np.intersect1d(sample_ids, cramps_nlp_all)
dryskin_nlp = np.intersect1d(sample_ids, dryskin_nlp_all)
fatigue_nlp = np.intersect1d(sample_ids, fatigue_nlp_all)
itching_nlp = np.intersect1d(sample_ids, itching_nlp_all)
musclesore_nlp = np.intersect1d(sample_ids, musclesore_nlp_all)
# Calculate true positive for each symptom by intersection between NLP and actual detections
tp_cramps = len(np.intersect1d(cramps_nlp, cramps_actual))
tp_dryskin = len(np.intersect1d(dryskin_nlp, dryskin_actual))
tp_fatigue = len(np.intersect1d(fatigue_nlp, fatigue_actual))
tp_itching = len(np.intersect1d(itching_nlp, itching_actual))
tp_musclesore = len(np.intersect1d(musclesore_nlp, musclesore_actual))
# Calculate false positives
fp_cramps = len(np.setdiff1d(cramps_nlp, cramps_actual))
fp_dryskin = len(np.setdiff1d(dryskin_nlp, dryskin_actual))
fp_fatigue = len(np.setdiff1d(fatigue_nlp, fatigue_actual))
fp_itching = len(np.setdiff1d(itching_nlp, itching_actual))
fp_musclesore = len(np.setdiff1d(musclesore_nlp, musclesore_actual))
# Calculate true negatives
tn_cramps = len(np.intersect1d(np.setdiff1d(sample_ids, cramps_nlp), np.setdiff1d(sample_ids, cramps_actual)))
tn_dryskin = len(np.intersect1d(np.setdiff1d(sample_ids, dryskin_nlp), np.setdiff1d(sample_ids, dryskin_actual)))
tn_fatigue = len(np.intersect1d(np.setdiff1d(sample_ids, fatigue_nlp), np.setdiff1d(sample_ids, fatigue_actual)))
tn_itching = len(np.intersect1d(np.setdiff1d(sample_ids, itching_nlp), np.setdiff1d(sample_ids, itching_actual)))
tn_musclesore = len(np.intersect1d(np.setdiff1d(sample_ids, musclesore_nlp), np.setdiff1d(sample_ids, musclesore_actual)))
# Calculate false negatives
fn_cramps = len(np.setdiff1d(np.setdiff1d(sample_ids, cramps_nlp), np.setdiff1d(sample_ids, cramps_actual)))
fn_dryskin = len(np.setdiff1d(np.setdiff1d(sample_ids, dryskin_nlp), np.setdiff1d(sample_ids, dryskin_actual)))
fn_fatigue = len(np.setdiff1d(np.setdiff1d(sample_ids, fatigue_nlp), np.setdiff1d(sample_ids, fatigue_actual)))
fn_itching = len(np.setdiff1d(np.setdiff1d(sample_ids, itching_nlp), np.setdiff1d(sample_ids, itching_actual)))
fn_musclesore = len(np.setdiff1d(np.setdiff1d(sample_ids, musclesore_nlp), np.setdiff1d(sample_ids, musclesore_actual)))

print(tp_cramps, tp_dryskin, tp_fatigue, tp_itching, tp_musclesore)
print(fp_cramps, fp_dryskin, fp_fatigue, fp_itching, fp_musclesore)
print(tn_cramps, tn_dryskin, tn_fatigue, tn_itching, tn_musclesore)
print(fn_cramps, fn_dryskin, fn_fatigue, fn_itching, fn_musclesore)
# Aggregate
tp = tp_cramps + tp_dryskin + tp_fatigue + tp_itching + tp_musclesore
fp = fp_cramps + fp_dryskin + fp_fatigue + fp_itching + fp_musclesore
tn = tn_cramps + tn_dryskin + tn_fatigue + tn_itching + tn_musclesore
fn = fn_cramps + fn_dryskin + fn_fatigue + fn_itching + fn_musclesore

print(tp, fp, tn, fn)
# Calculate metrics
accuracy = (tp+tn) / (tp+fp+tn+fn)
precision = tp / (tp+fp)
sensitivity = tp / (tp+fn)
specificity = tn / (tn+fp)
ppv = tp / (tp+fp)
npv = tn / (tn+fn)

# print(accuracy, precision, 
print('sensitivity:', sensitivity, ', specificity:', specificity)
print('PPV:', ppv, ', NPV:', npv)


In [ ]:
# investigate large number of false negatives

fn_cramps_ids = np.setdiff1d(np.setdiff1d(sample_ids, cramps_nlp), np.setdiff1d(sample_ids, cramps_actual))
fn_dryskin_ids = np.setdiff1d(np.setdiff1d(sample_ids, dryskin_nlp), np.setdiff1d(sample_ids, dryskin_actual))
fn_fatigue_ids = np.setdiff1d(np.setdiff1d(sample_ids, fatigue_nlp), np.setdiff1d(sample_ids, fatigue_actual))
fn_itching_ids = np.setdiff1d(np.setdiff1d(sample_ids, itching_nlp), np.setdiff1d(sample_ids, itching_actual))
fn_musclesore_ids = np.setdiff1d(np.setdiff1d(sample_ids, musclesore_nlp), np.setdiff1d(sample_ids, musclesore_actual))

print('FN cramps: ', fn_cramps_ids)
print('FN dry skin: ', fn_dryskin_ids)
print('FN fatigue: ', fn_fatigue_ids)
print('FN itching: ', fn_itching_ids)
print('FN muscle soreness: ', fn_musclesore_ids)


In [ ]:
# Display instances of false negatives
pd.set_option('display.max_colwidth', None)

print('CRAMPS')
for study_id in fn_cramps_ids:
    study_id_filt = df_match_v2['Study ID'] == study_id
    symptom_filt = df_match_v2['Symptom'] == 'Cramps'
    note_ids = df_match_v2.loc[study_id_filt & symptom_filt, 'Note ID'].str[1:-1].astype(float).to_list()
    note_filt = df_notes['Note ID'].isin(note_ids)
    display(df_notes.loc[note_filt & (df_notes['Study ID']==study_id), ['Study ID', 'Note ID', 'Progress note']])
    
print('FATIGUE')
for study_id in fn_fatigue_ids:
    study_id_filt = df_match_v2['Study ID'] == study_id
    symptom_filt = df_match_v2['Symptom'] == 'Fatigue'
    note_ids = df_match_v2.loc[study_id_filt & symptom_filt, 'Note ID'].str[1:-1].astype(float).to_list()
    note_filt = df_notes['Note ID'].isin(note_ids)
    display(df_notes.loc[note_filt & (df_notes['Study ID']==study_id), ['Study ID', 'Note ID', 'Progress note']])
    
print('ITCHING')
for study_id in fn_itching_ids:
    study_id_filt = df_match_v2['Study ID'] == study_id
    symptom_filt = df_match_v2['Symptom'] == 'itching'
    note_ids = df_match_v2.loc[study_id_filt & symptom_filt, 'Note ID'].str[1:-1].str.split(',').to_list()[0]
    note_ids = [float(nid) for nid in note_ids]
    note_filt = df_notes['Note ID'].isin(note_ids)
    display(df_notes.loc[note_filt & (df_notes['Study ID']==study_id), ['Study ID', 'Note ID', 'Progress note']])
    
print('MUSCLE SORENESS')
for study_id in fn_musclesore_ids:
    study_id_filt = df_match_v2['Study ID'] == study_id
    symptom_filt = df_match_v2['Symptom'] == 'Muscle Soreness'
    note_ids = df_match_v2.loc[study_id_filt & symptom_filt, 'Note ID'].str[1:-1].str.split(',').to_list()[0]
    note_ids = [float(nid) for nid in note_ids]
    note_filt = df_notes['Note ID'].isin(note_ids)
    display(df_notes.loc[note_filt & (df_notes['Study ID']==study_id), ['Study ID', 'Note ID', 'Progress note']])
      

In [ ]:
# investigate large number of false positives

fp_cramps_ids = np.setdiff1d(cramps_nlp, cramps_actual)
fp_dryskin_ids = np.setdiff1d(dryskin_nlp, dryskin_actual)
fp_fatigue_ids = np.setdiff1d(fatigue_nlp, fatigue_actual)
fp_itching_ids = np.setdiff1d(itching_nlp, itching_actual)
fp_musclesore_ids = np.setdiff1d(musclesore_nlp, musclesore_actual)


print('FP cramps: ', fp_cramps_ids)
print('FP dry skin: ', fp_dryskin_ids)
print('FP fatigue: ', fp_fatigue_ids)
print('FP itching: ', fp_itching_ids)
print('FP muscle soreness: ', fp_musclesore_ids)
